In [ ]:
import os
import json
import cv2
import mediapipe as mp
import numpy as np
import time

In [ ]:
mp_holistic = mp.solutions.holistic

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 3)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468 * 3)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    return np.concatenate([pose, face, left_hand, right_hand])

def load_missing_video_ids(missing_file):
    with open(missing_file, 'r') as f:
        missing_ids = {line.strip() for line in f}
    return missing_ids

def process_metadata(metadata_file, video_base_path, save_base_path, missing_ids, frame_skip=2):
    start_time = time.time() 
    with open(metadata_file, 'r') as f:
        metadata = json.load(f)

    total_videos = 11980  
    processed_videos = 0

    for entry in metadata:
        gloss = entry['gloss']
        for instance in entry['instances']:
            split = instance['split']
            video_id = instance['video_id']

            
            if video_id in missing_ids:
                print(f"Skipping missing video: {video_id}")
                continue

            video_file = os.path.join(video_base_path, f"{video_id}.mp4")

            # Ensure video exists
            if not os.path.exists(video_file):
                print(f"Video not found: {video_file}")
                continue

            
            save_dir = os.path.join(save_base_path, split, gloss)
            os.makedirs(save_dir, exist_ok=True)

            # Process video and extract keypoints
            keypoints = []
            cap = cv2.VideoCapture(video_file)
            with mp_holistic.Holistic(
                static_image_mode=False, 
                min_detection_confidence=0.5, 
                min_tracking_confidence=0.5,
                model_complexity=2,  
                enable_segmentation=True  
            ) as holistic:
                frame_count = 0  

                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break
                    
                    frame_count += 1
                    
                    if frame_count % frame_skip != 0:
                        continue

                    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    image.flags.writeable = False

                    results = holistic.process(image)

                    keypoint = extract_keypoints(results)
                    keypoints.append(keypoint)

            cap.release()

            np.savez_compressed(os.path.join(save_dir, f"{video_id}.npz"), keypoints=np.array(keypoints))

            processed_videos += 1
            elapsed_time = time.time() - start_time

            # Log progress
            print(f"Processed {processed_videos}/{total_videos} videos: {video_id}.npz")
            print(f"Elapsed time: {elapsed_time:.2f} seconds")

    total_time = time.time() - start_time
    print(f"Processing complete! Total time: {total_time:.2f} seconds")
    print(f"Total videos processed: {processed_videos}/{total_videos}")


In [ ]:
# Paths
metadata_file = "WLASL_v0.3.json"
video_base_path = "videos"
save_base_path = "video_key_points"
missing_file = "wlasl_class_list.txt"

missing_ids = load_missing_video_ids(missing_file)
process_metadata(metadata_file, video_base_path, save_base_path, missing_ids, frame_skip=2)
